# Disaster Tweets Classification

Summary: Goal is to determine which tweets are about real disasters or not
Dataset: https://www.kaggle.com/competitions/nlp-getting-started/leaderboard

The goal of this initial workbook is to complete the EDA to further investigate the data and potential features within the dataset.

## Lib Imports

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string

In [81]:
from nltk.corpus import stopwords
import preprocessor as p

ENGLISH_STOPWORDS = set(stopwords.words('english'))

In [82]:
RANDOM_STATE = 12
SUBMISSION_VERSION = 'v2'

## Train / Test Dataset Loading and Analysis

**Train Dataset Analysis**

In [83]:
train_df = pd.read_csv('train.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [84]:
train_df.isna().sum() * 100 / len(train_df)

id           0.000000
keyword      0.801261
location    33.272035
text         0.000000
target       0.000000
dtype: float64

Seem to be missing alot of values from keywords and location within the training dataset.

In [85]:
test_df = pd.read_csv('test.csv')
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


Now I am going to investigate whether there are any differences in distribution of target class between the missing and non missing location subsets of the training dataframe.

In [86]:
train_df[train_df['location'].isna()]['target'].value_counts(normalize=True)

target
0    0.575602
1    0.424398
Name: proportion, dtype: float64

In [87]:
train_df[~train_df['location'].isna()]['target'].value_counts(normalize=True)

target
0    0.567717
1    0.432283
Name: proportion, dtype: float64

It doesn't look like there are any substantial differences, which is quite positive.

**Test Dataframe Analysis**

In [88]:
test_df.isna().sum() * 100 / len(test_df)

id           0.000000
keyword      0.796813
location    33.864542
text         0.000000
dtype: float64

The testing dataset has similar missing data, whereby over 33% of locations are missing from the dataset.

In [89]:
train_df['target'].value_counts(normalize=True)

target
0    0.57034
1    0.42966
Name: proportion, dtype: float64

### Keyword Analysis

Initial observation is that keyword represents a single word or 'category' for the tweet.

In [90]:
missing_keyword_mask = (train_df['keyword'].isna())

In [91]:
keyword_train = train_df[~missing_keyword_mask]
keyword_train

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0


In [92]:
keyword_train['keyword'].value_counts()

keyword
fatalities               45
deluge                   42
armageddon               42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: count, Length: 221, dtype: int64

### Location Analysis

In [93]:
missing_location_mask = (train_df['location'].isna())

In [94]:
location_train = train_df[~missing_location_mask]
location_train.head()

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0


In [95]:
location_train['location'].value_counts()

location
USA                    104
New York                71
United States           50
London                  45
Canada                  29
                      ... 
MontrÌ©al, QuÌ©bec       1
Montreal                 1
ÌÏT: 6.4682,3.18287      1
Live4Heed??              1
Lincoln                  1
Name: count, Length: 3341, dtype: int64

There seems to be quite a mixture of different types of text within the location column, with some entries just being the country, some containing the city and country, and others not being a real location.

In [96]:
contains_comma_mask = (location_train['location'].str.find(',') != -1)

In [97]:
location_train[contains_comma_mask]['location'].value_counts()

location
Los Angeles, CA                26
Washington, DC                 21
Chicago, IL                    18
California, USA                15
New York, NY                   15
                               ..
Birdland, New Meridian, FD      1
Monterrey, MÌ©xico              1
2B Hindhede Rd, Singapore       1
Kalimantan Timur, Indonesia     1
Vancouver, Canada               1
Name: count, Length: 1305, dtype: int64

I'm going to need to find a better way of analysing the makeup of this location column....

### Text Analysis

After some investigation and review of other tweets, it is apparent the the text can contains any of the below features:

* 1. tags: any text starting with '#' will contain tags
* 2. user_tags: any text starting with @ represents a user tag
* 3. urls: text starting with https://, http:// etc contain urls
* 4. retweets: contains t.co?


In [98]:
def preprocess_text(df):
    def remove_punctuations(text: str):
        for punctuation in string.punctuation:
            text = text.replace(punctuation, '')
        return text

    df['text'] = df['text'].str.lower()
    #df['text'] = df['text'].str.replace('[^\w\s]', ' ')
    df['text'] = df['text'].apply(remove_punctuations)
    df['text'] = df['text'].str.replace('\s\s+', ' ')
    df['text'] = df['text'].apply(p.clean)
    df['text'] = df['text'].apply(lambda x : ' '.join([w for w in x.split(' ') if w not in ENGLISH_STOPWORDS]))


In [99]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [100]:
preprocess_text(train_df)

In [101]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,deeds reason earthquake may allah forgive us,1
1,4,NaN,NaN,forest fire near la ronge sask canada,1
2,5,NaN,NaN,residents asked shelter place notified officer...,1
3,6,NaN,NaN,people receive wildfires evacuation orders cal...,1
4,7,NaN,NaN,got sent photo ruby alaska smoke wildfires pou...,1


## X, y and train_test_split

In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

tfid_vectorizer = TfidfVectorizer()

In [103]:
X = tfid_vectorizer.fit_transform(train_df['text'])
y = train_df['target']

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=RANDOM_STATE, test_size=0.3)

## Logistic Regression Training and Evaluation

In [105]:
from sklearn.metrics import classification_report

In [106]:
from sklearn.linear_model import LogisticRegressionCV

logistic_model = LogisticRegressionCV(
    Cs=6,
    cv=2,
    scoring='f1'
)

In [107]:
logistic_model.fit(X_train, y_train)

LogisticRegressionCV(Cs=6, cv=2, scoring='f1')

In [108]:
y_pred = logistic_model.predict(X_test)

In [109]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.87      0.83      1336
           1       0.78      0.67      0.72       948

    accuracy                           0.79      2284
   macro avg       0.78      0.77      0.77      2284
weighted avg       0.78      0.79      0.78      2284



## Kaggle Submission

In [110]:
test_df = pd.read_csv('test.csv', delimiter=',')

In [111]:
len(test_df)

3263

In [112]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [113]:
test_df[test_df['id'] == 60]['text']

19    PSA: IÛªm splitting my personalities.\n\n?? t...
Name: text, dtype: object

In [114]:
preprocess_text(test_df)

In [115]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,happened terrible car crash
1,2,NaN,NaN,heard earthquake different cities stay safe ev...
2,3,NaN,NaN,forest fire spot pond geese fleeing across str...
3,9,NaN,NaN,apocalypse lighting spokane wildfires
4,11,NaN,NaN,typhoon soudelor kills china taiwan


In [116]:
X_val = tfid_vectorizer.transform(test_df['text'])

In [117]:
X_val

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 24313 stored elements and shape (3263, 21729)>

In [118]:
y_pred_val = logistic_model.predict(X_val)

In [119]:
y_pred_val

array([1, 1, 1, ..., 1, 1, 1])

In [120]:
submission_df = pd.DataFrame({'id': test_df['id'], 'target': y_pred_val})

In [121]:
submission_df.to_csv(f"submission_{SUBMISSION_VERSION}.csv", index=False)